# Yelp Analysis

For this notebook segment, I used the yelp API to determine where the 'hot and new' businesses are located. My rationale was that the areas where newer companies are located are most likely to be areas that are being gentrified. Unfortunately, there weren't that many results. Currently waiting on my Twitter API to see if I can access mobility patterns in different areas, which is a good indicator of gentrification:

https://www.npr.org/sections/13.7/2017/08/29/546980178/what-does-it-take-to-see-gentrification-before-it-happens


In [47]:
import requests
import pandas as pd
from config import api_key

In [48]:
# Business Search URL
url = 'https://api.yelp.com/v3/businesses/search'

In [49]:
# Define the api key, define the endpoint and define the header
API_KEY = api_key
ENDPOINT = url
HEADERS = {'Authorization': 'bearer %s' % yelp_key}
# headers is how we authenticate ourselves

In [50]:
# Define the parameters
PARAMETERS = {'categories': 'coffee, restaurants, bars',
             'radius': 40000,
             'location': 'Austin',
             'attributes': 'hot_and_new'}

In [51]:
# Make a request to the yelp API
response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)

# Convert JSON string to dictionary
biz_data = response.json()
print(biz_data)

{'businesses': [{'id': 't8MhdpBVeL2YxIUXx1kcfw', 'alias': 'ani-s-day-and-night-austin', 'name': 'Ani’s Day & Night', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/umrzV4DJR0vQDQdtM6ciig/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/ani-s-day-and-night-austin?adjust_creative=1ost9EOUCmQzzm2BRxkQWQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1ost9EOUCmQzzm2BRxkQWQ', 'review_count': 6, 'categories': [{'alias': 'foodtrucks', 'title': 'Food Trucks'}, {'alias': 'bars', 'title': 'Bars'}, {'alias': 'coffee', 'title': 'Coffee & Tea'}], 'rating': 4.5, 'coordinates': {'latitude': 30.22101, 'longitude': -97.6956}, 'transactions': [], 'location': {'address1': '7107 E Riverside Dr', 'address2': '', 'address3': None, 'city': 'Austin', 'zip_code': '78741', 'country': 'US', 'state': 'TX', 'display_address': ['7107 E Riverside Dr', 'Austin, TX 78741']}, 'phone': '+15127751355', 'display_phone': '(512) 775-1355', 'distance': 11097.854408045292}, {'id': '82

In [52]:
for biz in biz_data['businesses']:
    print(biz['location'])

{'address1': '7107 E Riverside Dr', 'address2': '', 'address3': None, 'city': 'Austin', 'zip_code': '78741', 'country': 'US', 'state': 'TX', 'display_address': ['7107 E Riverside Dr', 'Austin, TX 78741']}
{'address1': '1814 Harvey St', 'address2': None, 'address3': '', 'city': 'Austin', 'zip_code': '78702', 'country': 'US', 'state': 'TX', 'display_address': ['1814 Harvey St', 'Austin, TX 78702']}
{'address1': '3309 W State Hwy 29', 'address2': None, 'address3': '', 'city': 'Georgetown', 'zip_code': '78628', 'country': 'US', 'state': 'TX', 'display_address': ['3309 W State Hwy 29', 'Georgetown, TX 78628']}


In [66]:
df_nested_list = pd.json_normalize(biz_data)
df_nested_list

,businesses,total,region.center.longitude,region.center.latitude
0,"[{'id': 't8MhdpBVeL2YxIUXx1kcfw', 'alias': 'an...",3,-97.757721,30.305156


In [70]:
df_nested_list = pd.json_normalize(biz_data, record_path =['businesses'])
df_nested_list

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,t8MhdpBVeL2YxIUXx1kcfw,ani-s-day-and-night-austin,Ani’s Day & Night,https://s3-media3.fl.yelpcdn.com/bphoto/umrzV4...,False,https://www.yelp.com/biz/ani-s-day-and-night-a...,6,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,[],...,30.221010,-97.695600,7107 E Riverside Dr,,None,Austin,78741,US,TX,"[7107 E Riverside Dr, Austin, TX 78741]"
1,82mMA_TH0GQM3WYgxKXREA,zeds-real-fruit-ice-cream-austin,Zeds Real Fruit Ice Cream,https://s3-media1.fl.yelpcdn.com/bphoto/yhOH81...,False,https://www.yelp.com/biz/zeds-real-fruit-ice-c...,3,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",3.5,[],...,30.282100,-97.705730,1814 Harvey St,None,,Austin,78702,US,TX,"[1814 Harvey St, Austin, TX 78702]"
2,BRsU4qwL3hfdT6D5RbY2uA,eat-my-biscuits-georgetown,Eat My Biscuits,https://s3-media2.fl.yelpcdn.com/bphoto/HuHpnP...,False,https://www.yelp.com/biz/eat-my-biscuits-georg...,17,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"[delivery, pickup]",...,30.630148,-97.745067,3309 W State Hwy 29,None,,Georgetown,78628,US,TX,"[3309 W State Hwy 29, Georgetown, TX 78628]"


In [71]:
df_nested_list = df_nested_list.drop(['id','alias','image_url',
                                      'display_phone','categories', 
                                      'is_closed','url','rating',
                                      'transactions','location.address2',
                                      'location.address3'], axis=1)

In [72]:
df_nested_list

,name,review_count,phone,distance,coordinates.latitude,coordinates.longitude,location.address1,location.city,location.zip_code,location.country,location.state,location.display_address
0,Ani’s Day & Night,6,+15127751355,11097.854408,30.221010,-97.695600,7107 E Riverside Dr,Austin,78741,US,TX,"[7107 E Riverside Dr, Austin, TX 78741]"
1,Zeds Real Fruit Ice Cream,3,+15033955038,5614.057431,30.282100,-97.705730,1814 Harvey St,Austin,78702,US,TX,"[1814 Harvey St, Austin, TX 78702]"
2,Eat My Biscuits,17,+15125911129,36157.748173,30.630148,-97.745067,3309 W State Hwy 29,Georgetown,78628,US,TX,"[3309 W State Hwy 29, Georgetown, TX 78628]"


# Housing Market Analysis by zipcode

source: https://data.austintexas.gov/Housing-and-Real-Estate/2014-Housing-Market-Analysis-Data-by-Zip-Code/hcnj-rei3

In [75]:
df = pd.read_csv('2014_Housing_Market_Analysis_Data_by_Zip_Code.csv')
df.dropna()
df

,Zip Code,Population below poverty level,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",...,Rental units affordable to average teacher,Owner units affordable to average teacher,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related
0,78726.0,9,66096,16,20,3,4,2,0.0,1,...,70,0.0,99,17.0,375,20,55,2,786,40
1,NaN,19,52431,16,35,8,7,8,24.0,10,...,74,22.0,98,58.0,65,31,74,29,668,38
2,78724.0,38,35711,29,63,8,8,28,85.0,16,...,78,84.0,100,96.0,208,37,46,21,786,40
3,78617.0,18,43957,12,67,10,15,23,NaN,11,...,63,NaN,99,NaN,101,74,21,16,865,42
4,78701.0,20,68152,16,14,10,9,0,7.0,7,...,29,7.0,90,30.0,12,115,59,97,433,23
5,78702.0,33,34734,18,56,14,11,10,21.0,41,...,80,15.0,99,67.0,3,73,207,96,590,39
6,78703.0,10,92606,9,9,6,4,4,3.0,11,...,51,2.0,92,13.0,7,65,104,67,629,25
7,78704.0,21,50248,7,30,9,7,3,13.0,12,...,76,13.0,99,33.0,33,40,126,76,629,33
8,78705.0,66,11917,21,17,4,14,2,39.0,8,...,57,38.0,94,79.0,49,70,40,100,511,30
9,78717.0,3,93305,25,14,4,5,9,7.0,1,...,70,6.0,98,67.0,638,2,33,3,708,47


# ATX housing data

limitations: 
- most recent version from earlier this year so some of these properties are no longer available and the prices are not accurate

In [77]:
df = pd.read_csv('austinHousingData.csv')
df

,zpid,city,streetAddress,zipcode,description,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,...,numOfMiddleSchools,numOfHighSchools,avgSchoolDistance,avgSchoolRating,avgSchoolSize,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,homeImage
0,111373431,pflugerville,14424 Lake Victor Dr,78660,"14424 Lake Victor Dr, Pflugerville, TX 78660 i...",30.430632,-97.663078,1.98,2,True,...,1,1,1.266667,2.666667,1063,14,3.0,4,2,111373431_ffce26843283d3365c11d81b8e6bdc6f-p_f...
1,120900430,pflugerville,1104 Strickling Dr,78660,Absolutely GORGEOUS 4 Bedroom home with 2 full...,30.432673,-97.661697,1.98,2,True,...,1,1,1.400000,2.666667,1063,14,2.0,4,1,120900430_8255c127be8dcf0a1a18b7563d987088-p_f...
2,2084491383,pflugerville,1408 Fort Dessau Rd,78660,Under construction - estimated completion in A...,30.409748,-97.639771,1.98,0,True,...,1,1,1.200000,3.000000,1108,14,2.0,3,1,2084491383_a2ad649e1a7a098111dcea084a11c855-p_...
3,120901374,pflugerville,1025 Strickling Dr,78660,Absolutely darling one story home in charming ...,30.432112,-97.661659,1.98,2,True,...,1,1,1.400000,2.666667,1063,14,2.0,3,1,120901374_b469367a619da85b1f5ceb69b675d88e-p_f...
4,60134862,pflugerville,15005 Donna Jane Loop,78660,Brimming with appeal & warm livability! Sleek ...,30.437368,-97.656860,1.98,0,True,...,1,1,1.133333,4.000000,1223,14,3.0,3,2,60134862_b1a48a3df3f111e005bb913873e98ce2-p_f.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15166,29512934,austin,905 Silcantu Dr,78748,Not for rent!\n\nBeautiful one story home in T...,30.161997,-97.816345,1.98,6,False,...,1,1,0.900000,3.333333,1424,14,2.0,3,1,29512934_ff9b6eefa7e2eb4e9ef983da13a23098-p_f.jpg
15167,241937773,austin,1413 Waterloo Shore Ln UNIT 12,78741,This contemporary three-story detached condo h...,30.240223,-97.772835,1.98,2,True,...,1,1,1.900000,6.666667,1226,16,4.0,3,3,241937773_66d3e483bd783eac5a52ff5f938d2a2e-p_f...
15168,29473281,austin,2206 S 3rd St,78704,Original bungalow with a modern addition. Larg...,30.244835,-97.761124,1.98,2,False,...,1,1,0.800000,5.000000,889,13,3.0,3,2,29473281_9e90ec4652c4b3b6592a7fdd09f1ea6d-p_f.jpg
15169,29392029,austin,2000 Chestnut Ave,78722,So many options at this address in the eclecti...,30.281569,-97.717789,1.98,0,False,...,1,1,1.300000,6.666667,1057,15,2.0,4,1,29392029_a9a8306ea363d23f37d91d37975a1b96-p_f.jpg


In [80]:
df.zipcode.value_counts()

78748    1144
78745    1039
78749     774
78704     706
78737     617
78739     605
78732     597
78759     561
78723     558
78757     523
78717     516
78731     495
78744     472
78753     442
78727     417
78702     397
78758     395
78754     393
78747     381
78750     321
78729     318
78735     311
78703     302
78721     255
78730     244
78728     225
78741     224
78724     223
78733     214
78746     195
78726     178
78751     174
78736     163
78725     162
78752     153
78756     140
78722     100
78617      86
78705      68
78660      36
78619      12
78734       9
78701       8
78719       5
78653       4
78652       4
78742       4
78738       1
Name: zipcode, dtype: int64